<a href="https://colab.research.google.com/github/AreebAhmad-02/Real-time-rag-pipeline-for-rag-research-papers/blob/main/Fine_tuning_embedding_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install distilabel --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.8/290.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [17]:
!pip install --upgrade -qqq git+https://github.com/argilla-io/distilabel.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
%pip install -q -U "distilabel[hf-inference-endpoints, argilla]"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.8 MB/s eta 0:00:00


In [18]:
!pip install datasets -q -U

In [10]:
!pip install huggingface-hub -q -U

# Creating hugging face dataset and data cleaning

In [19]:
import os
import json
from datasets import Dataset

## data cleaning

In [ ]:

# Directory containing JSON files
directory = r'D:\rag prac\rag papers rag pipeline\chunks'

# Dictionary to hold data from all JSON files


def remove_newline_and_pipe(input_string):
    cleaned_string = input_string.replace('\n', ' ').replace('|', '')
    return cleaned_string


combined_data = []
system_prompt =  '''Provided the following context, I want to generate QA embedding pairs for each of my classes that are 3B and 7B LLM based on increasing complexity. For the 3B model, generate simple question-answer pairs, and for the 7B model, generate slightly more complex pairs. Please format the output in JSON as: {3B: {q1: ..., a1: ...}, {q2: ..., a2: ...}}, {7B: {q1: ..., a1: ...}, {q2: ..., a2: ...}}'''
# Iterate over all files in the directory
for filename in os.listdir(directory):
    # print(filename)
    if filename.endswith('.json'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r',encoding='utf-8') as file:
            print('filename:--------------', filename)
            data = json.load(file)
            clean_data = remove_newline_and_pipe(data['text'])
            print(type(clean_data))
            print("clean data:-----------------", clean_data)

            combined_data.append(
                {"text": clean_data, "system_prompt": system_prompt})

# Write the combined dictionary to a new JSON file
output_filepath = os.path.join(directory, 'output/combined.json')
with open(output_filepath, 'w') as output_file:
    json.dump(combined_data, output_file, indent=1)

print(f'Combined JSON saved to {output_filepath}')


In [4]:
with open('combined.json','r') as file:
  data_chunks_and_prompts = json.load(file)

In [8]:

ds = Dataset.from_list(data_chunks_and_prompts)


In [12]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [14]:
ds.push_to_hub("30ragpapersdataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Areeb-02/30ragpapersdataset/commit/6c05b530f4c8f8bcf8bb30642d3f03a6f2aa376e', commit_message='Upload dataset', commit_description='', oid='6c05b530f4c8f8bcf8bb30642d3f03a6f2aa376e', pr_url=None, pr_revision=None, pr_num=None)

# generating data set using distilabel

In [20]:
#importing dataset
from datasets import load_dataset
dataset = load_dataset("Areeb-02/30ragpapersdataset")
dataset

Generating train split:   0%|          | 0/1232 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'system_prompt'],
        num_rows: 1232
    })
})

In [31]:
# storing a mistral api key in environment using getpass code

from getpass import getpass

mistral_api_key = getpass('Enter your key:')
os.environ['MISTRAL_API_KEY'] = mistral_api_key


Enter your key:··········


In [21]:
from distilabel.llms import MistralLLM
from distilabel.pipeline import Pipeline
from distilabel.steps import CombineColumns, LoadHubDataset
from distilabel.steps.tasks import TextGeneration


In [29]:
!pip install mistralai -q


In [32]:
with Pipeline(name = "qa-text-generation-pipeline", description="QA text generation pipeline") as pipeline:
    load_dataset = LoadHubDataset(

        output_mappings={"text": "instruction"},  #
    )
    task = TextGeneration( llm=MistralLLM(model="mistral-large-2402", api_key = mistral_api_key))
    load_dataset.connect(task)



<ipython-input-32-d174d373b505>:2: DeprecationWarning: `LoadHubDataset` is deprecated and will be removed in version 1.3.0, use `LoadDataFromHub` instead.
  load_dataset = LoadHubDataset(


In [33]:


    if __name__ == "__main__":
      distiset = pipeline.run(
          parameters={
              load_dataset.name: {
                  "repo_id": "Areeb-02/30ragpapersdataset",
                  "split": "train",
              },
              task.name: {
                  "llm": {
                      "generation_kwargs": {
                          "temperature": 0.2,
                          "max_new_tokens": 512,
                      }
                  }
              },
          },
      )

[07/03/24 02:09:15] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to                ]8;id=874835;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=732385;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#611\611]8;;\
                             '/root/.cache/distilabel/pipelines/qa-text-generation-pipeline/7d3f6cac901            
                             572aa3a64991f4eefb1cd415f9fba/data'                                                   

[07/03/24 02:09:16] INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps to load...            ]8;id=596708;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=911129;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#703\703]8;;\

[07/03/24 02:09:18] INFO     ['distilabel.pipeline'] ⏳ Steps loaded: 2/2                               ]8;id=697155;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=578226;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#728\728]8;;\
                              * 'load_hub_dataset_0' workers: 1                                                    
                              * 'text_generation_0' workers: 1                                                     

                    INFO     ['distilabel.pipeline'] ✅ All the steps have been loaded!                 ]8;id=890832;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=52973;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#732\732]8;;\

                    INFO     ['distilabel.step.load_hub_dataset_0'] 🧬 Starting yielding batches from  ]8;id=955604;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=314942;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py#452\452]8;;\
                             generator step 'load_hub_dataset_0'. Offset: 0                                        

...

                    WARNING  ['distilabel.step.text_generation_0'] Subprocess traceback:               ]8;id=728804;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=360886;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py#528\528]8;;\
                                                                                                                   
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/httpx/_transports/default.py",               
                             line 69, in map_httpcore_exceptions                                                   
                                 yield                                                                             
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/httpx/_transports/default.py",               
                             line 373, in handle_async_request                                                     
                                 resp = await self._pool.handle_async_request(req)                                 
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/httpcore/_async/connection_pool.             
                             py", line 216, in handle_async_request                                                
                                 raise exc from None                                                               
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/httpcore/_async/connection_pool.             
                             py", line 196, in handle_async_request                                                
                                 response = await connection.handle_async_request(                                 
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/httpcore/_async/connection.py",              
                             line 101, in handle_async_request                                                     
                                 return await self._connection.handle_async_request(request)                       
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/httpcore/_async/http11.py", line             
                             143, in handle_async_request                                                          
                                 raise exc                                                                         
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/httpcore/_async/http11.py", line             
                             93, in handle_async_request                                                           
                                 await self._send_request_headers(**kwargs)                                        
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/httpcore/_async/ht

                    INFO     ['distilabel.step.text_generation_0'] 📨 Step 'text_generation_0' sending ]8;id=652912;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=434627;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py#558\558]8;;\
                             batch 24 to output queue                                                              

                    INFO     ['distilabel.step.text_generation_0'] 🏁 Finished running step            ]8;id=208438;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=177385;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py#414\414]8;;\
                             'text_generation_0'                                                                   

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [39]:
distiset['default']['train'][2]

{'instruction': 'Step 4: Query and Answer Generation. In this step, we leverage the bridge-entity or bridge-topic to generate multi-hop queries. Specifically, we first group the claims having the same bridge-entity or',
 'system_prompt': 'Provided the following context, I want to generate QA embedding pairs for each of my classes that are 3B and 7B LLM based on increasing complexity. For the 3B model, generate simple question-answer pairs, and for the 7B model, generate slightly more complex pairs. Please format the output in JSON as: {3B: {q1: ..., a1: ...}, {q2: ..., a2: ...}}, {7B: {q1: ..., a1: ...}, {q2: ..., a2: ...}}',
 'generation': None,
 'model_name': None}

In [ ]:


llm = InferenceEndpointsLLM(
    endpoint_name_or_model_id=endpoint_name,
    endpoint_namespace=endpoint_namespace,
    token=token,
    task=TextGenerationTask(),
    max_new_tokens=512,
    prompt_format="notus",
)
result = llm.generate([{"input": "What are critique LLMs?"}])
# print(result[0][0]["parsed_output"]["generations"])
# Critique LLMs (Long Land Moore Machines) are artificial intelligence models designed specifically for analyzing and evaluating the quality or worth of a particular subject or object. These models can be trained on a large dataset of reviews, ratings, or commentary related to a product, service, artwork, or any other topic of interest.
# The training data can include both positive and negative feedback, helping the LLM to understand the nuanced aspects of quality and value. The model uses natural language processing (NLP) techniques to extract meaningful insights, including sentiment analysis, entity recognition, and text classification.
# Once the model is trained, it can be used to analyze new input data and provide a critical assessment based on its learned understanding of quality and value. For example, a critique LLM for movies could evaluate a new film and generate a detailed review highlighting its strengths, weaknesses, and overall rating.
# Critique LLMs are becoming increasingly useful in various industries, such as e-commerce, education, and entertainment, where they can provide objective and reliable feedback to help guide decision-making processes. They can also aid in content optimization by highlighting areas of improvement or recommending strategies for enhancing user engagement.
# In summary, critique LLMs are powerful tools for analyzing and evaluating the quality or worth of different subjects or objects, helping individuals and organizations make informed decisions with confidence.